<img width="8%" alt="PyPI.jpeg" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/PyPI.jpeg" style="border-radius: 15%">

# PyPI - Get release dates from package
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=PyPI+-+Get+release+dates+from+package:+Error+short+description">Bug report</a>

**Tags:** #pypi #downloads #package #operations #analytics #plotly #html #csv #image #png

**Author:** [Mardiat-Iman](https://www.linkedin.com/in/mardiat-iman-ibrahim-imam-726027262)

**Last update:** 2023-07-27 (Created: 2023-07-27)

**Description:** This notebook get the release dates a package from the Python Package Index (PyPI) and plot a Barchart and Scatter Plot to display the release by month. 

NB: We have noticed that sometimes not all versions are accessible via this endpoint in comparison with the website. Please let us know if you manage to find a solution to this issue, we would appreciate.

## Input

### Import libraries

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from dateutil.parser import parse

### Setup variables

In [ ]:
# Allow the user to input the package name
user_input = input("Enter the package name to fetch release dates from PyPI: ")

# Extract the package name (remove version number if present)
package_name = user_input.split()[0]
print('Package:', package_name)

## Model

### Get releases dates

In [ ]:
def get_release_dates(package_name):
    # Init 
    release_dates = []
    
    # Request
    url = f"https://pypi.org/pypi/{package_name}/json"
    response = requests.get(url)
    
    # Response
    if response.status_code == 200:
        data = response.json()
        releases = data.get("releases", {})
        release_dates = [parse(release_info[0]["upload_time"]) for release_info in releases.values() if release_info]
        if not releases:
            print(f"No releases found for package '{package_name}'.")
        else:
            for version, release_info in releases.items():
                if release_info and isinstance(release_info, list):
                    latest_release = max(release_info, key=lambda x: x["upload_time"])
                    release_date = latest_release["upload_time"]
                    print(f"Version: {version}, Release Date: {release_date}")
    else:
        print(f"Error: Package '{package_name}' not found or error in fetching data.")
        print("Raw JSON Response:", response.text)
    return release_dates

# Call the function with the package name to get the release dates
release_dates = get_release_dates(package_name)
print("Release dates found:", len(release_dates))

## Output

### Plotting a Barchart and Scatter Plot for the release dates

In [ ]:
# Check if release dates were obtained successfully
if release_dates:
    # 1. Sort release dates in ascending order
    release_dates.sort()

    # 2. Bar Chart - Releases per Month
    release_months = [date.strftime('%Y-%m') for date in release_dates]
    unique_months, month_counts = np.unique(release_months, return_counts=True)
    unique_months = [parse(month + '-01') for month in unique_months]  # Convert back to datetime objects
    fig, ax = plt.subplots(figsize=(30, 15), constrained_layout=True)
    ax.set(title=f"Bar Chart - Releases per Month for {package_name.capitalize()}")
    ax.bar(unique_months, month_counts, color="tab:red", width=20)
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    ax.set_xlabel("Month")
    ax.set_ylabel("Number of Releases")
    plt.xticks(rotation=45, ha="right")
    ax.spines[["top", "right"]].set_visible(False)
    plt.savefig("bar_chart.png")  # Save the figure
    plt.show()
else:
    print("No release dates found for the specified package.")

### Save the figure

In [ ]:
fig.savefig("bar_chart.png")